In [1]:
!pip install textattack[tensorflow,optional]

In [2]:
!pip install tensorflow==2.14

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 98.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboar

In [3]:
!textattack peek-dataset --dataset-from-huggingface glue^qnli

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
2023-12-17 14:36:29 INFO: Downloading default packages for language: en (English) ...
2023-12-17 14:36:44 INFO: Finished downloading models and saved to /root/stanza_resources.
2023-12-17 14:36:47.402654: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unabl

In [4]:
!textattack train --model-name-or-path distilbert-base-uncased --dataset glue^qnli --model-num-labels 4 --model-max-length 350 --per-device-train-batch-size 128 --num-epochs 3

2023-12-17 14:49:34.255884: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 14:49:34.255932: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 14:49:34.255965: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 14:49:35.368428: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-17 14:49:38.000633: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the 

In [ ]:
import numpy as np
import tensorflow as tf

import torch
import textattack
import transformers

from textattack import AttackArgs
from textattack.datasets import Dataset
from textattack import Attacker

In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
dataset = textattack.datasets.HuggingFaceDataset("glue", "qnli", split="train")

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

textattack: Loading datasets dataset glue, subset qnli, split train.


In [ ]:
train_dataset = textattack.datasets.HuggingFaceDataset("glue", "qnli", split="train")
eval_dataset = textattack.datasets.HuggingFaceDataset("glue", "qnli", split="test")

textattack: Loading datasets dataset glue, subset qnli, split train.
textattack: Loading datasets dataset glue, subset qnli, split test.


In [ ]:
attack = textattack.attack_recipes.CLARE2020.build(model_wrapper)


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attack_args = textattack.AttackArgs(
    num_examples=100,
    checkpoint_interval=5,
    checkpoint_dir="checkpoints"
)

In [ ]:
attacker = textattack.Attacker(attack, dataset, attack_args)

results = attacker.attack_dataset()
ppl = textattack.metrics.quality_metrics.Perplexity().calculate(results)
usem = textattack.metrics.quality_metrics.USEMetric().calculate(results)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapMaskedLM(
        (method):  bae
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.0005
      )
    (1): WordInsertionMaskedLM(
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.0
      )
    (2): WordMergeMaskedLM(
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.005
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.7
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 9 / 7 / 16:  16%|█▌        | 16/100 [04:54<25:44, 18.38s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   1%|          | 1/100 [00:20<33:09, 20.10s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Not_entailment (51%)]] --> [[[FAILED]]]

[[[[Question]]]]: When did the third Digimon series begin?
[[[[Sentence]]]]: Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese.




[Succeeded / Failed / Skipped / Total] 0 / 2 / 2 / 4:   4%|▍         | 4/100 [00:36<14:41,  9.19s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Not_entailment (51%)]] --> [[[FAILED]]]

[[[[Question]]]]: Which missile batteries often have individual launchers several kilometres from one another?
[[[[Sentence]]]]: When MANPADS is operated by specialists, batteries may have several dozen teams deploying separately in small sections; self-propelled air defence guns may deploy in pairs.


--------------------------------------------- Result 3 ---------------------------------------------
[[Not_entailment (51%)]] --> [[[SKIPPED]]]

[[[[Question]]]]: What two things does Popper argue Tarski's theory involves in an evaluation of truth?
[[[[Sentence]]]]: He bases this interpretation on the fact that examples such as the one described above refer to two things: assertions and the facts to which they refer.


--------------------------------------------- Result 4 ---------------------------------------------
[[Not_entailment (52%)]] -->

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
print(ppl)

{'avg_original_perplexity': 85.62, 'avg_attack_perplexity': 90.41}


In [1]:
print(usem)

NameError: ignored

In [ ]:
training_args = textattack.TrainingArgs(
    num_epochs=3,
    num_clean_epochs=1,
    num_train_adv_examples=100,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    query_budget_train=120,
    save_last=True,
    load_best_model_at_end=True
)

trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    train_dataset,
    eval_dataset,
    training_args
)
m=trainer.train()


textattack: Writing logs to ./outputs/2023-12-16-15-41-07-989275/train_log.txt.
textattack: Wrote original training args to ./outputs/2023-12-16-15-41-07-989275/training_args.json.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 104743
textattack:   Num epochs = 3
textattack:   Num clean epochs = 1
textattack:   Instantaneous batch size per device = 8
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 32
textattack:   Gradient accumulation steps = 4
textattack:   Total optimization steps = 9828
textattack: ==========================================================
textattack: Epoch 1
textattack: Running clean epoch 1/1
Loss 0.40891

Loss 0.32115: 100%|██████████| 13106/13106 [1:20:01<00:00,  2.73it/s]
textattack: Train accuracy: 90.68%
textattack: Eval accuracy: 0.00%
textattack: ==========================================================
textattack: Epoch 3
textattack: Attacking model to generate new adversarial training set...
[Succeeded / Failed / Skipped / Total] 100 / 186 / 11 / 297: 100%|██████████| 100/100 [1:05:53<00:00, 39.54s/it]
textattack: Total number of attack results: 297
textattack: Attack success rate: 34.97% [100 / 286]


Loss 0.25138: 100%|██████████| 13106/13106 [1:20:00<00:00,  2.73it/s]
textattack: Train accuracy: 96.01%
textattack: Eval accuracy: 0.00%


HFValidationError: ignored

In [ ]:
!pip install pyproject-toml

  Using cached language-check-1.1.tar.gz (33 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for language-check
  Running setup.py clean for language-check
Failed to build language-check
ERROR: Could not build wheels for language-check, which is required to install pyproject.toml-based projects


In [ ]:
!pip install language-tool-python

In [ ]:
import re

import language_tool_python
from tqdm import tqdm


In [ ]:
refs = []
hypos = []
refsFile='/content/drive/MyDrive/Colab Notebooks/refsFile'
hyposFile='/content/drive/MyDrive/Colab Notebooks/hypos'

In [ ]:
with open(refsFile, 'r', encoding='utf8') as f:
  for line in f:
    refs.append(line.split('\n')[1].strip())

In [ ]:
hypos.append("Bryan Finlay was often aided in his matches by a midget known mainly as Hornswoggle while in WWE, who hid under the ring and gave a shillelagh to Finlay to use on his opponent.")
hypos.append("What kind of aesthetics did housebuilders use?")
hypos.append("In addition back to social and cultural characteristics, what else is taken into account for race classification in the US census?")
hypos.append("What is Torrence's 1989 theory about agriculture that ties into tool kit variability ?")
hypos.append("Her most popular early Marvel heroes were collectively and colloquially known as what?")
hypos.append("Analysis by what municipalities detailed that municipality-based rankings may be inaccurate?")


In [ ]:
def process_string(string):
    string = re.sub("( )(\'[(m)(d)(t)(ll)(re)(ve)(s)])", r"\2", string)
    string = re.sub("(\d+)( )([,\.])( )(\d+)", r"\1\3\5", string)
    # U . S . -> U.S.
    string = re.sub("(\w)( )(\.)( )(\w)( )(\.)", r"\1\3\5\7", string)
    # reduce left space
    string = re.sub("( )([,\.!?:;)])", r"\2", string)
    # reduce right space
    string = re.sub("([(])( )", r"\1", string)
    string = re.sub("s '", "s'", string)
    # reduce both space
    string = re.sub("(')( )(\S+)( )(')", r"\1\3\5", string)
    string = re.sub("(\")( )(\S+)( )(\")", r"\1\3\5", string)
    string = re.sub("(\w+) (-+) (\w+)", r"\1\2\3", string)
    string = re.sub("(\w+) (/+) (\w+)", r"\1\2\3", string)
    # string = re.sub(" ' ", "'", string)
    return string

In [ ]:
print('Evaluating number of grammar error:')
tool = language_tool_python.LanguageTool('en-US')
grammar_diffs = []
for i in tqdm(range(len(refs))):
  grammar_ref = len(tool.check(process_string(refs[i])))
  grammar_hypo = len(tool.check(process_string(hypos[i])))
  grammar_diffs.append(grammar_hypo - grammar_ref)
  gramar_err = np.mean(grammar_diffs)
  print("number of grammar difference: ", gramar_err)
  print('\n')

Evaluating number of grammar error:


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpv7xeo763.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.
 50%|█████     | 3/6 [00:01<00:01,  1.98it/s]

number of grammar difference:  0.0


number of grammar difference:  0.0


number of grammar difference:  0.0




100%|██████████| 6/6 [00:02<00:00,  2.94it/s]

number of grammar difference:  0.0


number of grammar difference:  0.0


number of grammar difference:  0.0


